In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
import numpy as np
import matplotlib.pyplot as plt
import sklearn.ensemble as forest
import GPyOpt
import GPy

%reload_ext autoreload
%autoreload 2
%matplotlib inline

# Purpose
This notebook uses random forest to pull out feature-importance for a dataset (regression) and shows it to the user together with the best and worst parameters for the dataset.
This to give information about the dataset to easier take descisions or actions.

The notebook can be divided into the follloing parts
1. prepare data
2. find best parameters for the random-forest model using bayesian optimization
3. Present result from the random-forest regression.

# Prepare Data

In [10]:
filename = "/temp/Train.csv"           # dataframe containing all the data
t_col_name = 'SalePrice'         # name of the column we are going to predict
split_pct = 0.2             # size of verification dataset

In [8]:
df = pd.read_csv(filename)

In [9]:
df.head()

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,fiModelDesc,fiBaseModel,fiSecondaryDesc,fiModelSeries,fiModelDescriptor,...,Coupler,Coupler_System,Grouser_Tracks,Hydraulics_Flow,Track_Type,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls
0,1139246,66000,999089,3157,121,3.0,2004,68.0,Low,11/16/2006 0:00,521D,521,D,NaN,NaN,...,None or Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
1,1139248,57000,117657,77,121,3.0,1996,4640.0,Low,3/26/2004 0:00,950FII,950,F,II,NaN,...,None or Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
2,1139249,10000,434808,7009,121,3.0,2001,2838.0,High,2/26/2004 0:00,226,226,NaN,NaN,NaN,...,None or Unspecified,None or Unspecified,None or Unspecified,Standard,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1139251,38500,1026470,332,121,3.0,2001,3486.0,High,5/19/2011 0:00,PC120-6E,PC120,NaN,-6E,NaN,...,None or Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1139253,11000,1057373,17311,121,3.0,2007,722.0,Medium,7/23/2009 0:00,S175,S175,NaN,NaN,NaN,...,None or Unspecified,None or Unspecified,None or Unspecified,Standard,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
def split_data(df_x, df_y, pct):
    "Splits the data into train and validation set"
    "takes into account the paste-batch"
    " pct: 0-1 depending on how much that is in validation set"
    df_x.reset_index(inplace=True, drop=True)
    df_y.reset_index(inplace=True, drop=True)

    l = int(len(df_x) * pct)
    x_train = df_x.iloc[:-l]
    x_valid = df_x.iloc[-l:]
    y_train = df_y.iloc[:-l]
    y_valid = df_y.iloc[-l:]
    return x_train, y_train, x_valid, y_valid

In [12]:
df_y = df[t_col_name]
df_x = df.drop(t_col_name, axis=1)
x_train, y_train, x_valid, y_valid = split_data(df_x, df_y, split_pct)

# Helper functions to train RF
we use rmse as measurment for how good the regression is

In [13]:
import math
def rmse(x,y): 
    return math.sqrt(((x-y)**2).mean())

def print_score(m):
    res = [rmse(m.predict(x_train), y_train), 
           rmse(m.predict(x_valid), y_valid),
                m.score(x_train, y_train), 
                m.score(x_valid, y_valid)]
    if hasattr(m, 'oob_score_'): 
        extra = f' oob_score_:{m.oob_score_}'
    else:
        extra = ''
        
    #print(f'rmse t:{res[0]}, rmse v:{res[1]} score t:{res[2]} score v:{res[3]}'+extra)
    return res[1]

In [14]:
def set_rf_samples(n):
    """ Changes Scikit learn's random forests to give each tree a random sample of
    n random rows.
    """
    forest._generate_sample_indices = (lambda rs, n_samples:
        forest.check_random_state(rs).randint(0, n_samples, n))

def reset_rf_samples():
    """ Undoes the changes produced by set_rf_samples.
    """
    forest._generate_sample_indices = (lambda rs, n_samples:
        forest.check_random_state(rs).randint(0, n_samples, n_samples))

In [15]:
def run_rf_regressor(min_sample_leaf:int=4, n_estimators:int=100, sample_frac:float=0.2, max_features:float=0.5):
    """
    todo, set max num samples to speed up the classification
    """
    set_rf_samples(int(len(x_train)*sample_frac))
    m = RandomForestRegressor(n_estimators=n_estimators, 
                              n_jobs=-1,
                              oob_score=True, 
                              min_samples_leaf=min_sample_leaf,
                              max_features=max_features)
    m.fit(x_train, y_train)
    rmse_valid = print_score(m)
    return rmse_valid
    
    

In [26]:
a = %timeit -n1 -r1 -o run_rf_regressor()
print(a)

2.49 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
2.49 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [ ]:
10 sek? räcker det

In [27]:
a.average

2.4937008249999053

In [16]:
run_rf_regressor()

ValueError: could not convert string to float: 'Low'

In [11]:
def x_to_dict(x):
    d = {'min_sample_leaf':int(x[0]),'n_estimators':int(x[1]), 'sample_frac':x[2],'max_features':x[3] }
    return d

In [12]:
nfold = 3
def fit_rf(x):
    fs = np.zeros((x.shape[0],1))
    for i in range(x.shape[0]):   # usually we only get one row of parameters for each call
        min_sample_leaf = int(x[i,0])
        n_estimators = int(x[i, 1])
        sample_frac = x[i,2]
        max_features = x[i,3]     
        fs[i] = 0
        #print(**x_to_dict(x[i,:]))
        for n in range(nfold):  # run fit 3 times to get a more stable result
            fs[i] += run_rf_regressor(**x_to_dict(x[i]))
        fs[i] *= 1./nfold
        print(fs[i])
    return fs

In [13]:
# todo, how to handle large datasets to minize time of fitting.

# Bayesian optmization
to find best hyper parameters for the random forest

In [14]:
domain       =[{'name': 'min_sample_leaf',      'type': 'continuous', 'domain': (2., 10.)},
               {'name': 'n_estimators','type': 'continuous', 'domain': (10.,100.)},
               {'name': 'sample_frac',  'type': 'continuous', 'domain': (0.1, 0.9)},
               {'name': 'max_features','type' : 'continuous', 'domain': (0.1, 1.0)}]


# This might take several minuts

In [15]:
opt = GPyOpt.methods.BayesianOptimization(f = fit_rf,            # function to optimize       
                                          domain = domain,         # box-constraints of the problem
                                          acquisition_type ='LCB',       # LCB acquisition
                                          acquisition_weight = 0.2)   # Exploration exploitation

[28451.6409785]
[28427.08125874]
[28109.54658503]
[28649.49006245]
[28374.39025153]
The set cost function is ignored! LCB acquisition does not make sense with cost.


In [16]:
opt.run_optimization(max_iter=50)
opt.plot_convergence()

[27827.24479869]


KeyboardInterrupt: 

# Best result
Loggs of which parameters that was ues is located in opt.X and the result is located in opt.Y if further investigation is needed

In [ ]:
x_best = opt.X[np.argmin(opt.Y)]
print(x_to_dict(x_best))
    

In [ ]:
x_best = x_to_dict(x_best)

In [ ]:
set_rf_samples(int(len(x_train)*x_best['sample_frac']))
m = RandomForestRegressor(n_estimators=x_best['n_estimators'], 
                          n_jobs=-1,
                          oob_score=True, 
                          min_samples_leaf=x_best['min_sample_leaf'],
                          max_features=x_best['max_features'])
m.fit(x_train, y_train)
print_score(m)

In [ ]:
y = m.predict(x_valid)

In [ ]:
plt.figure(figsize=(10,7))
plt.plot(y_valid.values, y, '.b')
plt.title(f'{t_col_name} Predictions')
plt.xlabel(f'actual {t_col_name}')
plt.ylabel(f'predicted {t_col_name}')

# Feature Importance
The following parameters are the most important for correct classification

In [ ]:
def rf_feat_importance(m, df):
    return pd.DataFrame({'cols':df.columns, 'imp':m.feature_importances_}
                       ).sort_values('imp', ascending=False)

def plot_fi(fi): return fi.plot('cols', 'imp', 'barh', figsize=(12,7), legend=False)

In [ ]:
plot_fi(rf_feat_importance(m, x_valid))

In [ ]:
from sklearn.tree import export_graphviz
import graphviz
import IPython
import re

# Draw descision tree
This might not work very good if we have many features in our dataset, but we make a try anyway

In [ ]:
def draw_tree(t, df, size=10,ratio=0.6, precision=0):
    """ Draws a representation of a random forest in IPython.
    Parameters:
    -----------
    t: The tree you wish to draw
    df: The data used to train the tree. This is used to get the names of the features.
    """
    s=export_graphviz(t, out_file=None, feature_names=df.columns, filled=True,
                      special_characters=True, rotate=True, precision=precision)
    IPython.display.display(graphviz.Source(re.sub('Tree {',
       f'Tree {{ size={size}; ratio={ratio}', s)))

In [ ]:
draw_tree(m.estimators_[0], df_x, precision=1)

In [ ]:
from treeinterpreter import treeinterpreter as ti

In [ ]:
def plot_item_importance(idx):
    prediction, bias, contributions = ti.predict(m, x_valid.loc[idx].values.reshape(1, -1))
    idxs = np.argsort(contributions[0])
    f_row=[o for o in zip(x_valid.columns[idxs], x_valid.loc[idx][idxs], contributions[0][idxs])]
    dfr = pd.DataFrame(f_row, columns=['colname', 'value', 'contribution'])
    dfr['contribution'] = dfr['contribution'].astype('float')
    dfr.plot('colname', 'contribution', 'barh', figsize=(12,7), legend=False, title=f'actual: {y_valid.loc[idx]} pred:{prediction}');

In [ ]:
#todo, we should include error, prediction- actual, 

# Feature importance for top 3

In [ ]:
largest = y_valid.nlargest(3).index.values
for idx in largest:
    plot_item_importance(idx)

# Feature importance for low 3

In [ ]:
smallest = y_valid.nsmallest(3).index.values
for idx in smallest:
    plot_item_importance(idx)